In [10]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import torch
import os
from skimage import io, transform
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image
from torchsummary import summary
import pandas as pd
import time
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from embedding_creator import preprocess

import argparse


In [11]:
def boolean_string(s):
    if s not in {'False', 'True'}:
        raise ValueError('Not a valid boolean string')
    return s == 'True'

try: 
    parser = argparse.ArgumentParser(description='Arguments for VAER')

    parser.add_argument('EMB_DIM', action="store", type=int)
    parser.add_argument('ENC_HID_DIM', action="store", type=int)
    parser.add_argument('DEC_HID_DIM', action="store", type=int)
    parser.add_argument('ATTN_DIM', action="store", type=int)
    parser.add_argument('EMB_DROPOUT', action="store", type=float)
    parser.add_argument('NET_DROPOUT', action="store", type=float)
    parser.add_argument('LR', action="store", type=float)
    parser.add_argument('TEACHER', action="store", type=float)
    parser.add_argument('SAVE_RESULTS', action="store")
    
    parser.add_argument('EMB_SOURCE', default= 'glove.6B.300d.NILC.txt', action="store")
    parser.add_argument('EMB_PORTINARI',
                        default='../data/data-portinari/embedding_matrix_NILC.npy', action="store")
    parser.add_argument('REMOVE_ACCENTS', default=False, action="store", type=boolean_string)
    
    
    
    print(parser.parse_args())

    read = parser.parse_args()
    
    EMB_DIM = read.EMB_DIM
    ENC_HID_DIM = read.ENC_HID_DIM
    DEC_HID_DIM = read.DEC_HID_DIM
    ATTN_DIM = read.ATTN_DIM
    EMB_DROPOUT = read.EMB_DROPOUT
    NET_DROPOUT = read.NET_DROPOUT
    LR = read.LR
    TEACHER = read.TEACHER
    SAVE_RESULTS = read.SAVE_RESULTS
    EMB_SOURCE = read.EMB_SOURCE
    EMB_PORTINARI = read.EMB_PORTINARI
    REMOVE_ACCENTS = read.REMOVE_ACCENTS
    
    print(EMB_SOURCE, EMB_PORTINARI, REMOVE_ACCENTS)
    print(read)
    '''EMB_SOURCE = read.EMB_SOURCE
    EMB_PORTINARI = read.EMB_PORTINARI'''
    
except:
    
    #---------------ARGS-----------------#
    EMB_DIM = 128
    ENC_HID_DIM = 256
    DEC_HID_DIM = 256
    ATTN_DIM = 8
    EMB_DROPOUT = 0.5
    NET_DROPOUT = 0.5
    LR = 0.004
    TEACHER = 0.5
    SAVE_RESULTS = '../results/results-portinari/progress_text_experiment_00' 
    EMB_SOURCE = 'glove.6B.300d.NILC.txt'
    EMB_PORTINARI = '../data/data-portinari/embedding_matrix_NILC.npy'
    REMOVE_ACCENTS = False

ENC_EMB_DIM = EMB_DIM
DEC_EMB_DIM = EMB_DIM
ENC_HID_DIM = ENC_HID_DIM
DEC_HID_DIM = DEC_HID_DIM
ATTN_DIM = ATTN_DIM
ENC_DROPOUT = EMB_DROPOUT
DEC_DROPOUT = EMB_DROPOUT
NET_DROPOUT = NET_DROPOUT
TEACHER = TEACHER
LR = LR
SAVE_RESULTS = SAVE_RESULTS

EMBEDDING_DIM = 300
LOG_INTERVAL = 40 
MAX_LEN = 200
BATCH_SIZE = 16
N_EPOCHS = 20


usage: ipykernel_launcher.py [-h]
                             EMB_DIM ENC_HID_DIM DEC_HID_DIM ATTN_DIM
                             EMB_DROPOUT NET_DROPOUT LR TEACHER SAVE_RESULTS
                             EMB_SOURCE EMB_PORTINARI REMOVE_ACCENTS
ipykernel_launcher.py: error: argument EMB_DIM: invalid int value: '/home/danielprado/.local/share/jupyter/runtime/kernel-a3c90507-87b2-420b-90bd-42414cf86aeb.json'


In [3]:
embedding_matrix, portinari_idx, retratos_idx, resto_idx, word_index, index_word = preprocess(True, 
        embedding_source = EMB_SOURCE, embedding_portinari = EMB_PORTINARI, remove_accents = REMOVE_ACCENTS)

no_cuda = False
seed = 1
cuda = not no_cuda and torch.cuda.is_available()
torch.manual_seed(seed)
device = torch.device("cuda" if cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
torch.manual_seed(seed)

class PortinariDesc(Dataset):
    def __init__(self, data) -> None:
        super().__init__()
        self.data = data

    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return len(self.data)
    
#cria o dataset


Achou 526 retratos
Indexing word vectors.
Found 929607 word vectors.
Null word embeddings: 1
total words 538345
null words 2199
0.41 % de palavras não encontradas no emb


In [13]:
from sklearn import model_selection
train_retratos, val_retratos = model_selection.train_test_split(retratos_idx, test_size = 0.2, shuffle = True)
train_resto, val_resto = model_selection.train_test_split(resto_idx, test_size = 0.2, shuffle = True)


train = train_retratos + train_resto
val = val_retratos + val_resto

In [14]:
train_tensor = list(map(lambda x: torch.tensor(x), train))
val_tensor = list(map(lambda x: torch.tensor(x), val))

train_dataset = PortinariDesc(train_tensor)
val_dataset = PortinariDesc(val_tensor)

def pad_collate(batch):
    xx = batch
    x_lens = list(map(len, xx))

    xx_pad = pad_sequence(xx, batch_first=True, padding_value=0, )

    return xx_pad, x_lens

#embedding_torch = nn.Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1])

In [15]:
unknown_tokens = [i for i in range(len(embedding_matrix)) if np.sum(embedding_matrix[i]-embedding_matrix[-3]) == 0]
for tk in unknown_tokens:
    index_word[tk] = '<unk>'
for w, idx in word_index.items():
    if np.sum(embedding_matrix[idx])==0:
        word_index[w] = len(embedding_matrix)-1
palavras_desconhecidas = [w for w, idx in word_index.items()  if np.sum(embedding_matrix[idx]-embedding_matrix[-3]) == 0]
print(f'palavras desconhecidas: {len(palavras_desconhecidas)}')

palavras desconhecidas: 294


In [16]:
palavras_desconhecidas

['3',
 '4',
 'pixains',
 'esvoaçados',
 'papilha',
 'batéia',
 'payots',
 'mancho',
 '2',
 'batatudo',
 '1',
 'grisados',
 'semilongos',
 'hachuriados',
 'jabô',
 '¡reas',
 'antùnio',
 'composiá\x84o',
 'geometrizando',
 'hubrecht',
 '·rea',
 'itaguahy',
 'enfumadas',
 '5',
 'mouries',
 'endomingada',
 'pregamento',
 'maáaroca',
 'simétricamente',
 'esvoaçada',
 'esfumato',
 'isûsceles',
 'semicerrado',
 'antebraáo',
 'aquareladas',
 'zínea',
 'ranhaduras',
 'est·',
 '¡rea',
 'aquarelado',
 'kipás',
 '8',
 'retratato',
 'sanfonado',
 'chapèu',
 'falangeta',
 'tiberiade',
 'verùnica',
 'abatatado',
 'esfumaturas',
 'entintamento',
 'planoa',
 '21',
 'craquelê',
 'heptagon',
 '90',
 'ampuleta',
 'gabirobas',
 'pùr',
 'sgraffito',
 'entrefechado',
 'zíneas',
 'm\x84os',
 'extens\x84o',
 'tínues',
 '·rvore',
 'constratando',
 'balangandans',
 '1941',
 'boiserie',
 'beduino',
 'beduina',
 'craquelado',
 'hachuriado',
 'emaranhas',
 'geometrismos',
 'gontrau',
 'esvoaçadas',
 'carbrito',
 'r

In [17]:
#palavras_desconhecidas

In [18]:
z = torch.distributions.categorical.Categorical(torch.Tensor([[3, 0.0, 3], [0.3, 0.4, 0.3]]))

In [19]:
z.sample()

tensor([0, 0])

In [20]:
embedding_tensor = torch.Tensor(embedding_matrix).to(device)


train_data_loader = torch.utils.data.DataLoader(
    train_dataset, drop_last=True,
    batch_size = BATCH_SIZE, shuffle=True, collate_fn=pad_collate)

val_data_loader = torch.utils.data.DataLoader(
    val_dataset, drop_last=True,
    batch_size = BATCH_SIZE, shuffle=True, collate_fn=pad_collate)

In [21]:
import random
from typing import Tuple

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch import Tensor

from text_models import Encoder, Decoder, Attention, Seq2Seq

In [22]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import torch
import os
from skimage import io, transform
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image
from skimage.io import imread
from torchsummary import summary
import pandas as pd
import time
from text_models import *
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

import random
from typing import Tuple

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch import Tensor


class Encoder(nn.Module):
    def __init__(self,
                 pre_trained_embedding: float,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 dropout: float):
        super().__init__()
        
        self.embedding = nn.Embedding.from_pretrained(
            embeddings=pre_trained_embedding)
        self.emb_dim = pre_trained_embedding.shape[1]
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.dropout = dropout
            
        self.rnn = nn.GRU(self.emb_dim, enc_hid_dim, bidirectional = True)

        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self,
                enc_input: Tensor, 
                enc_input_len) -> Tuple[Tensor]:

        embedded = self.dropout(self.embedding(enc_input))
        
        embedded_pack = torch.nn.utils.rnn.pack_padded_sequence(embedded, 
            enc_input_len, enforce_sorted=False)
        
        #print(embedded_pack)
        outputs_packed, hidden = self.rnn(embedded_pack)
        #print(outputs_packed[0].shape)
        #print(outputs_packed[1].shape)
        #print(outputs_packed.shape)
        outputs, _ = torch.nn.utils.rnn.pad_packed_sequence(outputs_packed)
        #print(outputs.shape)
        
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
        #print(hidden.shape)
        
        return outputs, hidden

    
class Attention(nn.Module):
    def __init__(self,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 attn_dim: int):
        super().__init__()

        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim

        self.attn_in = (enc_hid_dim * 2) + dec_hid_dim

        self.attn = nn.Linear(self.attn_in, attn_dim)

    def forward(self,
                decoder_hidden: Tensor,
                encoder_outputs: Tensor) -> Tensor:

        src_len = encoder_outputs.shape[0]

        repeated_decoder_hidden = decoder_hidden.unsqueeze(1).repeat(1, src_len, 1)

        encoder_outputs = encoder_outputs.permute(1, 0, 2)

        energy = torch.tanh(self.attn(torch.cat((
            repeated_decoder_hidden,
            encoder_outputs),
            dim = 2)))

        attention = torch.sum(energy, dim=2)

        return F.softmax(attention, dim=1)


class Decoder(nn.Module):
    def __init__(self,
                 pre_trained_embedding: float,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 dropout: int,
                 attention: nn.Module):
        super().__init__()
        
        self.embedding = nn.Embedding.from_pretrained(
            embeddings=pre_trained_embedding)
        self.output_dim = pre_trained_embedding.shape[0]
        self.emb_dim = pre_trained_embedding.shape[1]
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.dropout = dropout
        self.attention = attention
        
        self.rnn = nn.GRU((enc_hid_dim * 2) + self.emb_dim, dec_hid_dim)

        self.out = nn.Linear(self.attention.attn_in + self.emb_dim, self.output_dim)

        self.dropout = nn.Dropout(dropout)


    def _weighted_encoder_rep(self,
                              decoder_hidden: Tensor,
                              encoder_outputs: Tensor) -> Tensor:

        a = self.attention(decoder_hidden, encoder_outputs)

        a = a.unsqueeze(1)
        #print('a shape = {}'.format(a.shape))
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        #print('encoder_outputs shape = {}'.format(encoder_outputs.shape))
        
        weighted_encoder_rep = torch.bmm(a, encoder_outputs)
        #print('weighted_encoder_rep shape = {}'.format(weighted_encoder_rep.shape))
        
        weighted_encoder_rep = weighted_encoder_rep.permute(1, 0, 2)
        #print('weighted_encoder_rep shape = {}'.format(weighted_encoder_rep.shape))
        
        return weighted_encoder_rep


    def forward(self,
                dec_input: Tensor,
                #dec_input_len: Tensor, 
                decoder_hidden: Tensor,
                encoder_outputs: Tensor) -> Tuple[Tensor]:

        dec_input = dec_input.unsqueeze(0)

        embedded = self.dropout(self.embedding(dec_input))
        #print(embedded.shape)
        #print(f'embedded shape = {embedded.shape}')
        weighted_h = self._weighted_encoder_rep(decoder_hidden,
                                                encoder_outputs)
        
        #print(f'weighted_h shape = {weighted_h.shape}')
        rnn_input = torch.cat((embedded, weighted_h), dim = 2)
        
        #print(rnn_input.shape)
        #print(decoder_hidden.unsqueeze(0).shape)
        output, decoder_hidden = self.rnn(rnn_input, decoder_hidden.unsqueeze(0))

        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted_h = weighted_h.squeeze(0)

        output = self.out(torch.cat((output,
                                     weighted_h,
                                     embedded), dim = 1))

        return output, decoder_hidden.squeeze(0)    

class Seq2Seq(nn.Module):
    def __init__(self,
                 encoder: nn.Module,
                 decoder: nn.Module,
                 device: torch.device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self,
                x: Tensor,
                x_l : Tensor,
                teacher_forcing_ratio: float = 0.5) -> Tensor:

        batch_size = x.shape[1]
        max_len = x.shape[0]
        vocab_size = self.decoder.output_dim

        outputs = torch.zeros(max_len-1, batch_size, vocab_size).to(self.device)
        
        encoder_outputs, hidden = self.encoder(x, x_l)

        # first input to the decoder is the <sos> token
        output = x[0,:]
        #print(f'first output shape = {output}')
        
        for t in range(0, max_len-1):
            output, hidden = self.decoder(output, hidden, encoder_outputs)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.max(1)[1]
            #torch.distributions.categorical.Categorical(torch.Tensor([0.1, 0.0, 0.8]))
            #print(output.shape)
            #categorical_output = torch.distributions.categorical.Categorical(output)
            #top1 = categorical_output.sample()
            #print(top1)
            #print(top1.shape)
            output = (x[t+1,:] if teacher_force else top1)
            #if teacher_force: 
            #    print(f'output teacher force = {output}')
            
        return outputs

print("")

In [23]:
'''ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ATTN_DIM = 64
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5'''



enc = Encoder(embedding_tensor, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT).to(device)
attn = Attention(ENC_HID_DIM, DEC_HID_DIM, ATTN_DIM).to(device)
dec = Decoder(embedding_tensor, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT,  attn).to(device)
model = Seq2Seq(enc, dec, device).to(device)

In [24]:
def init_weights(m: nn.Module):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)

model.apply(init_weights)
print("")

In [25]:

optimizer = optim.Adam(model.parameters(), lr=LR)

In [26]:
def count_parameters(model: nn.Module):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f'The model has {count_parameters(model):,} trainable parameters')


The model has 10,909,442 trainable parameters


In [27]:
criterion = nn.CrossEntropyLoss(ignore_index=0)

In [28]:
'''import math
import time

(x,x_l) = next(iter(train_data_loader))
print(x)
x = x.to(device)
x_l = torch.Tensor(x_l).to(device)
output = model(x.permute(1,0),x_l)'''

'import math\nimport time\n\n(x,x_l) = next(iter(train_data_loader))\nprint(x)\nx = x.to(device)\nx_l = torch.Tensor(x_l).to(device)\noutput = model(x.permute(1,0),x_l)'

In [29]:
import math
import time

def train(model: nn.Module,
          iterator: DataLoader,
          optimizer: optim.Optimizer,
          criterion: nn.Module,
          clip: float,
          teacher : float,
          epoch):
    
    epoch_loss = 0
    epoch_start = 0
    train_loss = 0
    start = time.time()
    
    model.train(True)
    
    for batch_idx, (x,x_l) in enumerate(iterator):
        
        x = x.to(device)
       
        x_l = torch.Tensor(x_l).to(device)
        
        #print(f"x = {x.shape}, x_l = {x_l.shape}")
        
        output = model(x.permute(1,0),x_l, teacher)
        #output = output[:,1:,:]
        flat_output = output.view(-1, output.shape[-1])
        
        optimizer.zero_grad()
    
        
        
        target = x[:,1:].permute(1,0).contiguous().view(-1)
        
        loss = criterion(flat_output, target)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

        
        if batch_idx % LOG_INTERVAL == 0:
                interval = time.time() - start
                start = time.time()
                epoch_start = epoch_start + interval

                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss : {:.6f} \tTime Interv: {:.6f}'.format(
                    epoch, batch_idx * len(x), len(iterator.dataset),
                           100. * batch_idx / len(iterator),
                           loss.item(), interval))
        #del(x)
        #del(x_l)
        #torch.cuda.empty_cache()
            
    return epoch_loss / len(iterator)

In [30]:
def evaluate(model: nn.Module,
             iterator: DataLoader,
             criterion: nn.Module):

    model.eval()

    epoch_loss = 0

    with torch.no_grad():

        for batch_idx, (x,x_l) in enumerate(iterator):

            x = x.to(device)

            x_l = torch.Tensor(x_l).to(device)
            
            output = model(x.permute(1,0),x_l, 0)
            #output = output[:,1:,:]
            
            flat_output = output.view(-1, output.shape[-1])
            target = x[:,1:].permute(1,0).contiguous().view(-1)

            loss = criterion(flat_output, target)

            epoch_loss += loss.item()
            del(x)
            del(x_l)
            torch.cuda.empty_cache()

    return epoch_loss / len(iterator)

In [31]:
def epoch_time(start_time: int,
               end_time: int):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


CLIP = 1

best_valid_loss = float('inf')
last_valid_loss = 100
valid_loss_patience = 100
valid_loss_register = []
early_stop_patience = 5
early_stop_register = []


#for epoch in range(N_EPOCHS):
for epoch in range(N_EPOCHS):
    start_time = time.time()
    
    train_loss = train(model, train_data_loader, optimizer, criterion, CLIP, TEACHER, epoch)
    valid_loss = evaluate(model, val_data_loader, criterion)
    
    early_stop_register.append(valid_loss-last_valid_loss)
    valid_loss_register.append(valid_loss)
    
    
    if epoch >= early_stop_patience + 3:
        if valid_loss_register[-3] + sum(early_stop_register[-2:]) > 1*valid_loss_register[-3]:
            print(f'ref loss at -3  {valid_loss_register[-3]}')
            print(f'last 2 variations: {early_stop_register[-2:]}')
            break
            
    if epoch >= early_stop_patience + 5:
        if valid_loss_register[-5] + sum(early_stop_register[-4:]) > 0.95*valid_loss_register[-5]:
            print(f'ref loss at -5 {valid_loss_register[-5]}')
            print(f'last 4 variations: {early_stop_register[-4:]}')
            break
            

    last_valid_loss = valid_loss
    
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')
    
try: 
    progress = np.load(SAVE_RESULTS + ".ndpy")
    progress = progress.tolist()
    progress.append((str(read), train_loss, valid_loss))
    np.save(SAVE_RESULTS, progress) 
except FileNotFoundError: 
    np.save(SAVE_RESULTS, []) 
except:
    print('unkown exception!')

Train Epoch: 0 [0/3630 (0%)]	Loss : 9.046227 	Time Interv: 2.731362
Train Epoch: 0 [640/3630 (18%)]	Loss : 6.051653 	Time Interv: 41.074471
Train Epoch: 0 [1280/3630 (35%)]	Loss : 5.897942 	Time Interv: 41.785329
Train Epoch: 0 [1920/3630 (53%)]	Loss : 5.897379 	Time Interv: 41.002971
Train Epoch: 0 [2560/3630 (71%)]	Loss : 5.726577 	Time Interv: 40.924388


KeyboardInterrupt: 

In [ ]:

print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

In [ ]:
#def print_sents():
with torch.no_grad():
    model.train()
    (x,x_l) = next(iter(train_data_loader))
    x = x.to(device)
    x_l_tens = torch.Tensor(x_l).to(device)
    output = model(x.permute(1,0),x_l_tens, 0)
    out = output.argmax(-1).transpose(1,0).cpu().numpy().tolist()
    inp = x.cpu().numpy().tolist()
    inp_text = [[index_word[token] if token != 0 else 'NUll' for token in sentence ] 
              for sentence in inp]
    out_text = [[index_word[token] if token != 0 else 'NUll' for token in sentence ] 
                  for sentence in out]

    print('\nTrain = True')
    print('\nfrase inp:', inp_text[0][0:x_l[0]+1])
    print('\nfrase out:',  out_text[0][0:x_l[0]+1])

    print('\nTrain = False')

    model.eval()
    output = model(x.permute(1,0),x_l_tens, 0)
    out = output.argmax(-1).transpose(1,0).cpu().numpy().tolist()
    inp = x.cpu().numpy().tolist()
    inp_text = [[index_word[token] if token != 0 else 'NUll' for token in sentence ] 
              for sentence in inp]
    out_text = [[index_word[token] if token != 0 else 'NUll' for token in sentence ] 
                  for sentence in out]
    print('\nfrase inp:', inp_text[0][0:x_l[0]+1])
    print('\nfrase out:',  out_text[0][0:x_l[0]+1])

#print_sents()

In [ ]:
#x[:,1:].permute(1,0).contiguous().view(-1).cpu().numpy().tolist()

In [ ]:
#output.view(-1, output.shape[-1]).max(1)[0].cpu().numpy().tolist()